In [1]:
from bs4 import BeautifulSoup
import requests
from multiprocessing import Pool

GIST_URL = "https://gist.github.com/paulmillr/2657075"

def get_soup_from_url(u):
    return BeautifulSoup(requests.get(u).content, 'html.parser')

In [2]:
def get_all_usernames(url):
    soup = get_soup_from_url(url)
    rows = soup.select('article table tbody tr')
    # The username is the text of the first link of each row:
    usernames = [row.find('a').text for row in rows]
    return usernames

In [3]:
def get_user_mean_stars(username):
    print(f'Retrieving infos for user {username}...')
    url = f'https://api.github.com/users/{username}/repos'
    headers = {'Authorization': 'token XXXXXXXXXXXXX'}
    resp = requests.get(url, headers=headers)
    if resp.status_code != 200:
        print('oops')
        print(resp.content)
    repos = resp.json()
    nb_repos = len(repos)
    if nb_repos == 0:
        return 0
    total_stars = sum(repo['stargazers_count'] for repo in repos)
    return total_stars / nb_repos

In [4]:
def get_user_infos(username):
    return {
        'name': username,
        'mean_stars': get_user_mean_stars(username),
    }

def top_users():
    all_usernames = get_all_usernames(GIST_URL)
    scores = [get_user_infos(u) for u in all_usernames]
    #p = Pool(processes=2)
    #scores = p.map(get_user_infos, all_usernames)
    return scores

In [5]:
users = pd.DataFrame(top_users())

Retrieving infos for user fabpot...
Retrieving infos for user andrew...
Retrieving infos for user taylorotwell...
Retrieving infos for user egoist...
Retrieving infos for user HugoGiraudel...
Retrieving infos for user ornicar...
Retrieving infos for user bebraw...
Retrieving infos for user nelsonic...
Retrieving infos for user alexcrichton...
Retrieving infos for user jonathanong...
Retrieving infos for user mikermcneil...
Retrieving infos for user benbalter...
Retrieving infos for user jxnblk...
Retrieving infos for user yegor256...
Retrieving infos for user orta...
Retrieving infos for user rstacruz...
Retrieving infos for user GrahamCampbell...
Retrieving infos for user afc163...
Retrieving infos for user kamranahmedse...
Retrieving infos for user joshaber...
Retrieving infos for user bkeepers...
Retrieving infos for user kennethreitz...
Retrieving infos for user kytrinyx...
Retrieving infos for user STRML...
Retrieving infos for user atmos...
Retrieving infos for user weierophinney

In [6]:
users.sort_values('mean_stars', ascending=False)

,name,mean_stars
18,kamranahmedse,4535.566667
225,bailicangdu,3254.210526
142,vinta,3214.333333
117,mrdoob,2227.533333
44,sebastianbergmann,2132.733333
...,...,...
149,gdi2290,0.000000
98,0x00A,0.000000
174,olivergierke,0.000000
122,paulcbetts,0.000000
